#Image Diff and Processing

I have included various approaches to the project idea.
Following are the proof of concepts for some features that I would like to implement and their corresponding python scripts which can be used for its implementation. 

Keeping in mind the whole idea of an image-analyzer in peragro-AT, there are various image processing features that can be included in the core peragro-AT, the image diff being the major one.

There are various image processing libraries which can be used to implement image diff, like python imaging library(PIL) being the most common one, scikit-image one of the most widely used tools for implementation of various type of analytic features, imgdiff and imagemagick library. Wand is a ctypes-based ImagedMagick binding library for python and pythonmagick is an object-oriented python interface to ImageMagick.

I have used scikit-image and PIL before and I am familiar with wand, so I would focus on implementing in skimage and PIL.

There are various features that we can implement in the peragro-AT's image analyzer along with image diff:
 * Image differencing, the major one
 * Similarity index between images(using various approaches like RMS, SSIM, MSE)
 * Extracting as much meta-data from images as possible
 * Other features such as image noising and denoising(improves quality, specially used for satellite images)
 * Blob detection(extreme bright or dark features in an image)
 * And ofcourse exception handling( It has to be everywhere, doesn't need much explanation )

I would like to know how do you perceive addtions of external features like denoising and blob detection. 


There can be more such features once the major ones are implemented.
Plugins can be made, so that whoever uses the analyzer has more control on the features they want in it. For example currently I have thought of implementing Blob detection as a plugin using the yapsy package, which is also currently used in peragro-AT. 

Keeping in mind the scope of the project and its future development, I would suggest using scikit-image as there are lot of image manipulations that can be done with it. Though most of the things which can be done using PIL can be done in scikit-image and both of theirs source code are majorly written in C, so they are likely to have insignificant performance differences. Thus I would go with using scikit-image along with numpy and other required modules so that in future, there is more scope of implementing various image processing features.



So to begin with, lets take the simplest feature:
To check the similarity between two images, we can easily do it by either PIL or skimage

In [ ]:
### USING ROOT-MEAN SQUARE DIFFERNCE TO CHECK SIMILARITY ###
def rmsdiff(im1, im2):
    h = ImageChops.difference(im1, im2).histogram()

    # calculate rms
    return math.sqrt(reduce(operator.add,
        map(lambda h, i: h*(i**2), h, range(256))
    ) / (float(im1.size[0]) * im1.size[1]))


Similarly skimage can used. Fortunately skimage has an inbuilt module for using structural similarity index. I have also implemented mean squared error. We can even use matplotlib to plot the required images.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import structural_similarity as ssim
## using openCV module ##
import cv2

pic = cv2.imread("satwik.png")
pic2 = cv2.imread("satwik2.png")
## for simplicity I am using grayscale here ##
pic = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
pic2 = cv2.cvtColor(pic2, cv2.COLOR_BGR2GRAY)


### MEAN SQUARED ERROR ###
def mse(im1, im2):
    err = np.sum((im1.astype("float") - im2.astype("float")) ** 2)
    err /= float(im1.shape[0] * im2.shape[1])
    return err
 
    
## STRUCTURAL SIMILARITY AND MEAN SQUARED ERROR
def similarity(imageA, imageB):
	s = ssim(im1, im2)
	m = mse(im1, im2)
	fig = plt.figure('satwik_test')
	plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
 
	ax = fig.add_subplot(1, 2, 1)
	plt.imshow(imageA, cmap = plt.cm.gray)
	plt.axis("off")
 
	ax = fig.add_subplot(1, 2, 2)
	plt.imshow(imageB, cmap = plt.cm.gray)
	plt.axis("off")

	plt.show()




Thus, we can use various tools to implement features, but I would prefer to stick to PIL and scikit-image, as I have worked on those previously and use one of them depending on which makes things easier to implement something.

To keep things neat and simple, in the following explanations I would include shorter code snippets to reduce complexity yet retain the root idea.



To get the diff between two images,
Using PIL

To get the general pixelwise diff: 

In [ ]:
def pixelwise_diff(im1,im2):
	red=abs(im1[0]-im2[0])
	green=abs(im1[1]-im2[1])
	blue=abs(im1[2]-im2[2])
	alpha = int(0.2*red + 0.2*green + 0.2*blue)
	return (red,green,blue,alpha)

For implementing it on a transparent image, we can use:

In [ ]:
im = Image.new("RGBA", (imagesize) , (0,0,0,0))

And then go through the same pixelwise_diff to get our required image.

To blend images we can use:

In [ ]:
Image.blend(image1, image2, alpha)

Though for all the above given implementations the image must be of the same size. Thus before using the above implementations the larger image has to be scaled to the smaller one or the smaller image could be filled with blank pixels. Both approaches can be given as options using argparse in command line. Images can be resized using:

In [ ]:
out = im.resize((height,width))


Further moving on to extracting meta-data from images, we need to get the exif tags from the images using PIL.

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS
 
def extract_exif(im1):
    result = {}
    im = Image.open(im1)
    info = im._getexif()
    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        result[decoded] = value
    return result

{'YResolution': (180, 1), 
 'ResolutionUnit': 2, 
 'Make': 'Canon', 
 'Flash': 16, 
 'DateTime': '2009:09:11 11:29:10', 
 'MeteringMode': 5, 
 'XResolution': (180, 1), 
 'ColorSpace': 1, 
 'ExifImageWidth': 3264, 
 'DateTimeDigitized': '2009:09:11 11:29:10', 
 'ApertureValue': (116, 32), 
 'FocalPlaneYResolution': (2448000, 169), 
 'CompressedBitsPerPixel': (3, 1), 
 'SensingMethod': 2, 
 'FNumber': (35, 10), 
 'DateTimeOriginal': '2009:09:11 11:29:10', 
 'FocalLength': (26000, 1000), 
 'FocalPlaneXResolution': (3264000, 225), 
 'ExifOffset': 196, 
 'ExifImageHeight': 2448, 
 'ISOSpeedRatings': 100, 
 'Model': 'Canon PowerShot S5 IS', 
 'Orientation': 1, 
 'ExposureTime': (1, 200), 
 'FileSource': '\x03', 
 'MaxApertureValue': (116, 32), 
 'ExifInteroperabilityOffset': 3346, 
 'FlashPixVersion': '0100', 
 'FocalPlaneResolutionUnit': 2, 
 'YCbCrPositioning': 1, 
 'ExifVersion': '0220'}


The exif tags can also be extracted using wand:

In [ ]:
exif = {}
with Image(filename='satwik.jpg') as image:
    exif.update((k[5:], v) for k, v in image.metadata.items()
                           if k.startswith('exif:'))

But for sake of reducing dependencies, I would refrain from using packages other than PIL and scikit-image unless necessary


Now coming over to external features like denoising, the concept is a bit complicated and lengthy. So in the process of removing noise yet preserving the details and structures, I would use Rudin-Osher-Fatemi de-noising model (ROF). It is relatively easy to implement using numpy. A simple implementation:

In [ ]:
from numpy import *

### An implementation of the Rudin-Osher-Fatemi (ROF) denoising model ###
def denoise(im,U_init,tolerance=0.1,tau=0.125,tv_weight=100):
  m,n = im.shape

  U = U_init
  Px = im 
  Py = im 
  error = 1

  while (error > tolerance):
    Uold = U
    GradUx = roll(U,-1,axis=1)-U 
    GradUy = roll(U,-1,axis=0)-U 

    PxNew = Px + (tau/tv_weight)*GradUx
    PyNew = Py + (tau/tv_weight)*GradUy
    NormNew = maximum(1,sqrt(PxNew**2+PyNew**2))

    Px = PxNew/NormNew 
    Py = PyNew/NormNew
    RxPx = roll(Px,1,axis=1)
    RyPy = roll(Py,1,axis=0)

    DivP = (Px-RxPx)+(Py-RyPy)
    U = im + tv_weight*DivP 
    error = linalg.norm(U-Uold)/sqrt(n*m);

  return U,im-U  

# returns denoised image and texture residual 

Similarly for blob detection, there are algorithms like laplacian of Gaussian(LoG), Difference of Gaussian(DoG) and Determinant of Gaussian(DoH) and for each of them, there are inbuilt libraries like:
 * skimage.feature.blob_log()
 * skimage.feature.blob_dog()
 * skimage.feature.blob_doh()

Thus it shows how using scikit-image, we can further extend our analyzer for implementation of various features. 

For implementing a GUI, Tkinter is a very use Python Interface in the Tk GUI toolkit inbuilt in python.